In [42]:
import pandas as pd
import numpy as np

In [43]:
voterfile = pd.read_csv('ST LOUIS COUNTY - ALL REG VOTERS_CSV.csv', dtype=str)

#Read in full county property file that includes all property types
countydata = pd.read_csv('countycombined.csv', dtype=str)

#Read in all residential units (SF and MF)
all_residential = pd.read_csv('sf_mf_units.csv', dtype=str)

#Read in single family csv only
sf_df = pd.read_csv('single_family_units.csv', dtype=str)

In [44]:
voterfile['full_address'] = voterfile['House Number'].str.strip() + ' ' + voterfile['Street Name'].str.strip() + ' ' + voterfile['Street Type'].str.strip()

#full_address will merge with TAXADR in the countydata file

In [45]:
merged = pd.merge(voterfile, sf_df, left_on='full_address', right_on='TAXADR', how='inner')

In [46]:
# This step will  perform matching between property owner name (single field) and voter name columns
# Next step: I need to normalize names by removing punctuation, uppercase, and strip whitespace
def normalize_series(s):
    return s.fillna('').astype(str).str.replace(r'[^\w\s]', '', regex=True).str.upper().str.strip()

# Create normalized owner and voter name fields
merged['OWNER_norm'] = normalize_series(merged['OWNNAME'])
owner_parts = merged['OWNER_norm'].str.split()
merged['owner_last_name'] = owner_parts.str[0].fillna('')
merged['owner_first_name'] = owner_parts.str[1].fillna('')

merged['voter_last_name_norm'] = normalize_series(merged['Last Name'])
merged['voter_first_name_norm'] = normalize_series(merged['First Name']).str.split().str[0].fillna('')

# Build matching indicators
merged['last_name_match'] = merged['voter_last_name_norm'] == merged['owner_last_name']
merged['first_name_match'] = merged['voter_first_name_norm'] == merged['owner_first_name']
merged['owner_voter_name_match'] = merged['last_name_match'] & merged['first_name_match']

# Extract matched DataFrames
matched_both = merged[merged['owner_voter_name_match']].copy()
matched_last_only = merged[merged['last_name_match'] & ~merged['owner_voter_name_match']].copy()

# Print counts and a small sample
print(f'Both-name matches: {len(matched_both)}')
print(f'Last-name-only matches: {len(matched_last_only)}')
print('Sample both-name matches (first 5 rows):')
print(matched_both[['full_address', 'OWNNAME', 'First Name', 'Last Name']].head())

# Note: if owner naming conventions vary (e.g., 'FIRST LAST' or include multiple owners),

Both-name matches: 212505
Last-name-only matches: 240795
Sample both-name matches (first 5 rows):
       full_address          OWNNAME First Name Last Name
1     9714 AMBRO LN   WARD MICHAEL C    MICHAEL      WARD
9   4074 DOMBARD LN    TOLES ORA MAE        ORA     TOLES
10  4075 DOMBARD LN  BARRERA HENRY J      HENRY   BARRERA
11  4078 DOMBARD LN    MCKINNEY EMMA       EMMA  MCKINNEY
13  4082 DOMBARD LN  DIRKS PATRICK J    PATRICK     DIRKS


In [47]:
merged.to_csv('merged_output.csv', index=False)

In [ ]:
#Divide tracts into low, medium, and high income levels
tracts = pd.read_csv('median_income_tracts.csv')
medianHH_2023 = 81340

#Low income tracts are $66064 and below
#Medium tracts are between $66065 and $102239
#High income tracts are between $102240 and above

# Define bins and labels
bins = [-np.inf, 66711, 104888, np.inf] # Define the boundaries for low, medium, high
labels = ['Low', 'Medium', 'High']

tracts['income_level'] = pd.cut(tracts['MedianIncome'], bins=bins, labels=labels, right=False)
tracts

,Geo__geoid_,Geo_qname,Geo_TRACT,MedianIncome,income_level
0,29189210101,"Census Tract 2101.01, St. Louis County, Missouri",210101,73438,Medium
1,29189210102,"Census Tract 2101.02, St. Louis County, Missouri",210102,55488,Low
2,29189210200,"Census Tract 2102, St. Louis County, Missouri",210200,38600,Low
3,29189210300,"Census Tract 2103, St. Louis County, Missouri",210300,55229,Low
4,29189210400,"Census Tract 2104, St. Louis County, Missouri",210400,48185,Low
...,...,...,...,...,...
231,29189221631,"Census Tract 2216.31, St. Louis County, Missouri",221631,107384,High
232,29189221800,"Census Tract 2218, St. Louis County, Missouri",221800,36351,Low
233,29189221900,"Census Tract 2219, St. Louis County, Missouri",221900,113261,High
234,29189222000,"Census Tract 2220, St. Louis County, Missouri",222000,82188,Medium


In [49]:
tracts.to_csv('tracts_with_income_levels.csv', index=False)

In [74]:
# For each address, prefer a voter who matched the owner (last_name_match==True).
# If no matched voter exists at that address, fall back to the first active voter.
# First keep only Active voters
active_voters = merged[merged['Voter Status'] == 'Active'].copy()
dropped_inactive = active_voters

#Print out the various entries under Voter Status to verify
print('Merged Voter Status values:', merged['Voter Status'].unique())

#Remove any remaining non-Active entries (safety - should be redundant)
dropped_inactive = dropped_inactive[dropped_inactive['Voter Status'] == 'Active']
print('After filter - dropped_inactive Voter Status values:', dropped_inactive['Voter Status'].unique())

#Save the active unique voters to CSV
dropped_inactive.to_csv('active_unique_voters.csv', index=False)

Merged Voter Status values: ['Active' 'Inactive' nan]
After filter - dropped_inactive Voter Status values: ['Active']
After filter - dropped_inactive Voter Status values: ['Active']


In [75]:
#Count the number of owner_voter_name_match true instances
num_owner_voter_matches = dropped_inactive['owner_voter_name_match'].sum()
num_last_name_matches = dropped_inactive['last_name_match'].sum()

print(f'Number of full matches in active voters: {num_owner_voter_matches}')
print(f'Number of last name matches in active voters: {num_last_name_matches}')

print(f'Full Name Match Ratio: {num_owner_voter_matches / len(dropped_inactive)}')
print(f'Last Name Match Ratio: {num_last_name_matches / len(dropped_inactive)}')

#BECAUSE LAST NAME MATCH IS CLOSEST TO GENERAL HOMEOWNERSHIP RATE IN STL COUNTY PER STL FRED (~72.9), UTILIZE LAST NAME MATCH

#Create a rentals column in the merged df for instances where all match types are false


Number of full matches in active voters: 208168
Number of last name matches in active voters: 440553
Full Name Match Ratio: 0.33026341048596797
Last Name Match Ratio: 0.698947659005345


In [78]:
#USE LAST NAME MATCH FOR DRILLDOWN INTO ONE UNIQUE ACTIVE VOTER PER ADDRESS
# For each address, prefer a voter who matched the owner (last_name_match==True).
# Ensure the match column exists; if not present create it and default to False

if 'last_name_match' not in active_voters.columns:
    active_voters['last_name_match'] = False

# Define chooser: prefer first row where last_name_match is True, else first row in group
def choose_preferred(group):
    matched = group[group['last_name_match'] == True]
    if not matched.empty:
        return matched.iloc[[0]]
    return group.iloc[[0]]

# Apply per-address selection (group_keys=False to return concatenated rows)
dropped_inactive = active_voters.groupby('full_address', group_keys=False).apply(choose_preferred).reset_index(drop=True)

C:\Users\brian\AppData\Local\Temp\ipykernel_16684\2378117706.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dropped_inactive = active_voters.groupby('full_address', group_keys=False).apply(choose_preferred).reset_index(drop=True)


In [79]:
#Create a rentals column in the merged df for instances where all match types are false
dropped_inactive['Rentals'] = dropped_inactive['last_name_match'] == False

In [80]:
#Create sample from tracts based on income levels and disproportionate stratified sampling
#First merge voters with census tracts and income signifiers

# Ensure both columns are int for merge
dropped_inactive['TRACTCE20'] = dropped_inactive['TRACTCE20'].astype(float).astype('Int64')
merged_voters_and_tracts = pd.merge(dropped_inactive, tracts, left_on='TRACTCE20', right_on='Geo_TRACT', how='inner')
merged_voters_and_tracts
#print length of dropped_inactive and merged_voters_and_tracts to verify no data loss
print(f'Length of dropped_inactive: {len(dropped_inactive)}')
print(f'Length of merged_voters_and_tracts: {len(merged_voters_and_tracts)}')

Length of dropped_inactive: 271059
Length of merged_voters_and_tracts: 271059


In [ ]:
#Count number of instances in each income level
#High: 109792
#Medium: 92281
#Low:  68986
#Total: 271059
#Desired sample size: 6000 total

sampling_fraction = 0.022142
low_sample = 3028
medium_sample = 1859
high_sample = 1113

merged_voters_and_tracts['income_level'].value_counts()

#Use sum of the low, medium, and high samples for actual sample size: 270,979

income_level
High      109792
Medium     92281
Low        68986
Name: count, dtype: int64

In [71]:
#Retrieve low, medium, and high income samples at desired sample sizes randomly
low_income_sample = merged_voters_and_tracts[merged_voters_and_tracts['income_level'] == 'Low'].sample(n=low_sample, random_state=1)
medium_income_sample = merged_voters_and_tracts[merged_voters_and_tracts['income_level'] == 'Medium'].sample(n=medium_sample, random_state=1)
high_income_sample = merged_voters_and_tracts[merged_voters_and_tracts['income_level'] == 'High'].sample(n=high_sample, random_state=1)

#Create new dataframe with all samples
final_sample = pd.concat([low_income_sample, medium_income_sample, high_income_sample])

#Check counts
final_sample['income_level'].value_counts()

#Count number of renters in final sample
num_renters_final_sample = final_sample['Rentals'].sum()
print(f'Number of renters in final sample: {num_renters_final_sample}') 
#Convert number of renters into percentage of sample   
renter_percentage = (num_renters_final_sample / len(final_sample)) * 100
print(f'Percentage of renters in final sample: {renter_percentage:.2f}%')
#Calculate homeownership rate as 1-renter_percentage
homeownership_rate = 100 - renter_percentage
print(f'Homeownership rate in final sample: {homeownership_rate:.2f}%')


Number of renters in final sample: 1536
Percentage of renters in final sample: 25.60%
Homeownership rate in final sample: 74.40%


In [72]:

#Export final sample to CSV
final_sample.to_csv('final_sample.csv', index=False)